In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

25/10/15 04:49:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## 1. Create namespace

In [2]:
%%sql
SHOW CATALOGS;

catalog
demo
spark_catalog


In [3]:
%%sql
SHOW DATABASES;

namespace
db


In [4]:
%%sql
CREATE NAMESPACE IF NOT EXISTS demo.db;

++
||
++
++

## 2. Read dataset

In [5]:
## checking the dataset presence
!ls dataset

insurance.csv


In [6]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("./dataset/insurance.csv")
df.show(5)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



In [7]:
df.createOrReplaceTempView("insurance_csv")

In [8]:
%%sql
SELECT * FROM insurance_csv
WHERE
  bmi = 39.82;

age,sex,bmi,children,smoker,region,charges
56,female,39.82,0,no,southeast,11090.7178
33,female,39.82,1,no,southeast,4795.6568
18,female,39.82,0,no,southeast,1633.9618


## 3. Partition

In [9]:
%%sql
CREATE
OR REPLACE TABLE demo.db.insurance USING iceberg PARTITIONED BY (region) AS
SELECT * FROM
  insurance_csv;

++
||
++
++

In [10]:
%%sql
SELECT * FROM demo.db.insurance LIMIT 5;

age,sex,bmi,children,smoker,region,charges
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552
37,female,27.74,3,no,northwest,7281.5056
60,female,25.84,0,no,northwest,28923.13692
37,male,28.025,2,no,northwest,6203.90175


## 3.1 Show partitions in the table

### Show the table definition

In [11]:
%%sql
DESCRIBE EXTENDED demo.db.insurance;

col_name,data_type,comment
age,int,None
sex,string,None
bmi,double,None
children,int,None
smoker,string,None
region,string,None
charges,double,None
# Partition Information,,
# col_name,data_type,comment
region,string,None


### Iceberg metadata tables

In [12]:
%%sql
SELECT * FROM demo.db.insurance.partitions;

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
Row(region='northwest'),1,325,1,6366,0,0,0,0,2025-10-15 04:49:50,6303910031349085422
Row(region='southwest'),1,325,1,5744,0,0,0,0,2025-10-15 04:49:50,6303910031349085422
Row(region='southeast'),1,364,1,6687,0,0,0,0,2025-10-15 04:49:50,6303910031349085422
Row(region='northeast'),1,324,1,6423,0,0,0,0,2025-10-15 04:49:50,6303910031349085422


### Run a partition‑pruning query

In [13]:
%%sql
EXPLAIN SELECT * FROM demo.db.insurance WHERE region = 'southeast';

plan
"== Physical Plan ==*(1) Filter (isnotnull(region#211) AND (region#211 = southeast))+- *(1) ColumnarToRow +- BatchScan demo.db.insurance[age#206, sex#207, bmi#208, children#209, smoker#210, region#211, charges#212] demo.db.insurance (branch=null) [filters=region IS NOT NULL, region = 'southeast', groupedBy=] RuntimeFilters: []"


## 4. Schema evolution

### 4.1 Add a new column

In [14]:
%%sql
ALTER TABLE demo.db.insurance
ADD COLUMN policy_id STRING;

++
||
++
++

In [15]:
%%sql
SELECT * FROM demo.db.insurance;

age,sex,bmi,children,smoker,region,charges,policy_id
33,male,22.705,0,no,northwest,21984.47061,None
32,male,28.88,0,no,northwest,3866.8552,None
37,female,27.74,3,no,northwest,7281.5056,None
60,female,25.84,0,no,northwest,28923.13692,None
37,male,28.025,2,no,northwest,6203.90175,None
55,female,32.775,2,no,northwest,12268.63225,None
23,male,17.385,1,no,northwest,2775.19215,None
63,male,28.31,0,no,northwest,13770.0979,None
19,male,20.425,0,no,northwest,1625.43375,None
62,female,32.965,3,no,northwest,15612.19335,None


### 4.2 Rename column

In [16]:
%%sql
ALTER TABLE demo.db.insurance
RENAME COLUMN sex TO gender;

++
||
++
++

In [17]:
%%sql
SELECT * FROM demo.db.insurance;

age,gender,bmi,children,smoker,region,charges,policy_id
33,male,22.705,0,no,northwest,21984.47061,None
32,male,28.88,0,no,northwest,3866.8552,None
37,female,27.74,3,no,northwest,7281.5056,None
60,female,25.84,0,no,northwest,28923.13692,None
37,male,28.025,2,no,northwest,6203.90175,None
55,female,32.775,2,no,northwest,12268.63225,None
23,male,17.385,1,no,northwest,2775.19215,None
63,male,28.31,0,no,northwest,13770.0979,None
19,male,20.425,0,no,northwest,1625.43375,None
62,female,32.965,3,no,northwest,15612.19335,None


### 4.3 Drop column

In [18]:
%%sql
ALTER TABLE demo.db.insurance
DROP COLUMN policy_id;

++
||
++
++

In [19]:
%%sql
SELECT * FROM demo.db.insurance;

age,gender,bmi,children,smoker,region,charges
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552
37,female,27.74,3,no,northwest,7281.5056
60,female,25.84,0,no,northwest,28923.13692
37,male,28.025,2,no,northwest,6203.90175
55,female,32.775,2,no,northwest,12268.63225
23,male,17.385,1,no,northwest,2775.19215
63,male,28.31,0,no,northwest,13770.0979
19,male,20.425,0,no,northwest,1625.43375
62,female,32.965,3,no,northwest,15612.19335


## 4.4 verify schema history

### Schema history

In [20]:
%%sql
SELECT * FROM demo.db.insurance.history ORDER BY made_current_at DESC;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-10-15 04:49:50,6303910031349085422,None,True
2025-10-15 04:30:49.209000,3858102109553479743,None,False
2025-10-15 04:15:21.270000,9164666238223732662,None,False
2025-10-15 04:11:20.838000,3561448738912927623,None,False


### Inspect metadata

In [21]:
%%sql
SELECT * FROM demo.db.insurance.snapshots;

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-10-15 04:11:20.838000,3561448738912927623,None,append,s3://warehouse/db/insurance/metadata/snap-3561448738912927623-1-07470707-e7dc-419e-b93a-b0edb2df5128.avro,"{'engine-version': '3.5.5', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1760501315703', 'added-records': '1338', 'total-records': '1338', 'spark.app.id': 'local-1760501315703', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '16528', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '16528', 'total-data-files': '1'}"
2025-10-15 04:15:21.270000,9164666238223732662,None,append,s3://warehouse/db/insurance/metadata/snap-9164666238223732662-1-e059a0fb-2052-4846-8358-10b7718246e6.avro,"{'engine-version': '3.5.5', 'added-data-files': '4', 'total-equality-deletes': '0', 'app-id': 'local-1760501710013', 'added-records': '1338', 'total-records': '1338', 'spark.app.id': 'local-1760501710013', 'changed-partition-count': '4', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '25220', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '25220', 'total-data-files': '4'}"
2025-10-15 04:30:49.209000,3858102109553479743,None,append,s3://warehouse/db/insurance/metadata/snap-3858102109553479743-1-ce0dac53-fc97-4aa5-803e-5c8eea6ec90c.avro,"{'engine-version': '3.5.5', 'added-data-files': '4', 'total-equality-deletes': '0', 'app-id': 'local-1760502640587', 'added-records': '1338', 'total-records': '1338', 'spark.app.id': 'local-1760502640587', 'changed-partition-count': '4', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '25220', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '25220', 'total-data-files': '4'}"
2025-10-15 04:49:50,6303910031349085422,None,append,s3://warehouse/db/insurance/metadata/snap-6303910031349085422-1-eb7d4f2e-7add-471a-aa07-50b8997a4816.avro,"{'engine-version': '3.5.5', 'added-data-files': '4', 'total-equality-deletes': '0', 'app-id': 'local-1760503776197', 'added-records': '1338', 'total-records': '1338', 'spark.app.id': 'local-1760503776197', 'changed-partition-count': '4', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '25220', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '25220', 'total-data-files': '4'}"


## 5. Time Travel
https://docs.databricks.com/gcp/en/delta/history

## 5.1 Query by snapshot ID

In [23]:
%%sql
SELECT * FROM demo.db.insurance.snapshot_id_3858102109553479743;

age,sex,bmi,children,smoker,region,charges
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552
37,female,27.74,3,no,northwest,7281.5056
60,female,25.84,0,no,northwest,28923.13692
37,male,28.025,2,no,northwest,6203.90175
55,female,32.775,2,no,northwest,12268.63225
23,male,17.385,1,no,northwest,2775.19215
63,male,28.31,0,no,northwest,13770.0979
19,male,20.425,0,no,northwest,1625.43375
62,female,32.965,3,no,northwest,15612.19335


## 5.2 Query by timestamp

In [24]:
%%sql
SELECT * FROM demo.db.insurance TIMESTAMP AS OF '2025-10-15 04:30:49.209000';

age,sex,bmi,children,smoker,region,charges
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552
37,female,27.74,3,no,northwest,7281.5056
60,female,25.84,0,no,northwest,28923.13692
37,male,28.025,2,no,northwest,6203.90175
55,female,32.775,2,no,northwest,12268.63225
23,male,17.385,1,no,northwest,2775.19215
63,male,28.31,0,no,northwest,13770.0979
19,male,20.425,0,no,northwest,1625.43375
62,female,32.965,3,no,northwest,15612.19335
